In [1]:
# PhraseMatcher.py
# import necessary modules

from __future__ import unicode_literals, print_function


from pathlib import Path
from spacy.util import minibatch, compounding
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy import displacy
from collections import Counter
from spacy.matcher import PhraseMatcher #import PhraseMatcher class
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
from time import sleep
from progressbar import progressbar
from spacy.tokens import Span
from spacy.util import minibatch, compounding
from collections import Counter

import nltk
import random
import plac
import warnings
import pandas as pd
import numpy as np
import xlrd
import spacy
import en_core_web_sm
import json 
import re


nltk.download('punkt')

# Language class with the English model 'en_core_web_sm' is loaded
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 7000000

[nltk_data] Downloading package punkt to /Users/abdelhak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Read the list of terminology 

In [2]:
terms_corpus = pd.read_excel('astronomy.xls')

In [3]:
# the list containing the pharses to be matched
terminology_list = []
for term in terms_corpus['key']:
    terminology_list.append(term[term.find(':')+2:])

### text for test

In [4]:
import glob
read_files = glob.glob("corpus/Astromony_*.txt")
with open("corpus/result.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())

# the input text string is converted to a Document object

file = open('corpus/result.txt')
text = file.read()

### Entity ruler

In [25]:
nlp_rule_based = English()
ruler = EntityRuler(nlp_rule_based)

In [26]:
# create patterns
patterns = []

for term in terminology_list:
    dct = {}
    temp = term.split()
    if len(temp) == 1:
        dct["label"] = "AstroTerm"
        dct["pattern"] = temp[0]
        patterns.append(dct)
    else:
        lst = []
        for item in temp:
            dct_temp = {}
            dct_temp["lower"] = item
            
            lst.append(dct_temp)
            
        dct["label"] = "AstroTerm"
        dct["pattern"] = lst
        patterns.append(dct)

In [27]:
ruler.add_patterns(patterns)
nlp_rule_based.add_pipe(ruler)

In [28]:
def extract_entities(doc):
    dict_new_ents = {}
    list_new_ents = []
    for ent in doc.ents:
        # Only check
        if ent.label_ == "AstroTerm":
            
            
            list_new_ents.append((ent.start_char, ent.end_char, ent.label_))      
    
    dict_new_ents['entities'] = list_new_ents
            
    return (doc.text,dict_new_ents )

In [17]:
train_data = []
for doc in nltk.tokenize.sent_tokenize(text):
    doc = nlp_rule_based(doc)
    train_data.append(extract_entities(doc))

In [20]:

def train_spacy(data, n_iter = 10, load = False):
    
    
    TRAIN_DATA = data
    # load space model
    if load:
        nlp = spacy.load("en_core_web_sm")
    else:
        nlp = spacy.blank('en')
        
   
        
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")
    
    # add labels
    # for _, annotations in TRAIN_DATA:
    #      for ent in annotations.get('entities'):
    ner.add_label('AstroTerm')
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    # Resume training
    #optimizer = nlp.resume_training()
    #move_names = list(ner.move_names)

    nlp.begin_training()

    # Begin training by disabling other pipeline components
    with nlp.disable_pipes(*other_pipes):
        # show warnings for misaligned entity spans once
        optimizer = nlp.begin_training()

        sizes = compounding(1.0, 5.0, 1.001)
        # Training for n_iter iterations     
        for itn in progressbar(range(n_iter)):
            sleep(0.02)
        # shuffle examples before training
            random.shuffle(TRAIN_DATA)
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=sizes)
            # dictionary to store losses
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                # Calling update() over the iteration
                nlp.update(texts, annotations, sgd=optimizer, drop=0.25, losses=losses)
                #print("Losses", losses)
    return nlp

In [21]:
nlp = train_spacy(train_data, 100)

/Users/abdelhak/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  **kwargs
100% (100 of 100) |#####################| Elapsed Time: 21:09:17 Time: 21:09:17


In [22]:
output_dir = "output_dir"
# if output_dir is not None:
#     output_dir = Path(output_dir)
#     if not output_dir.exists():
#         output_dir.mkdir()
#     nlp.to_disk(output_dir)
#     print("Saved model to", output_dir)

Saved model to output_dir


In [ ]:
# test the saved model
print("Loading from", output_dir)
nlp = spacy.load(output_dir)

In [30]:
file = open('test_corpus/text04.txt')
text_test = file.read()

In [37]:
doc = nlp(text_test)
entities = [(ent.text, ent.label_) for ent in doc.ents]
#print("Entities", entities)
counter = Counter(entities)
counter

Counter({('spectroscopic', 'AstroTerm'): 2,
         ('creativity', 'AstroTerm'): 1,
         ('happy', 'AstroTerm'): 1,
         ('expert', 'AstroTerm'): 1,
         ('cosmos', 'AstroTerm'): 1,
         ('galaxy', 'AstroTerm'): 2,
         ('interstellar gas', 'AstroTerm'): 1,
         ('dust', 'AstroTerm'): 1,
         ('star', 'AstroTerm'): 1,
         ('universe', 'AstroTerm'): 2,
         ('Black Hole', 'AstroTerm'): 1,
         ('black hole', 'AstroTerm'): 1,
         ('planet', 'AstroTerm'): 1,
         ('astronomy', 'AstroTerm'): 1,
         ('light', 'AstroTerm'): 2,
         ('collaboration', 'AstroTerm'): 1,
         ('telescope', 'AstroTerm'): 2,
         ('astrophysics', 'AstroTerm'): 1,
         ('Southern Hemisphere', 'AstroTerm'): 1})

In [38]:
doc = nlp_rule_based(text_test)
entities = [(ent.text, ent.label_) for ent in doc.ents]
#print("Entities", entities)
counter = Counter(entities)
counter

Counter({('spectroscopic', 'AstroTerm'): 2,
         ('happy', 'AstroTerm'): 1,
         ('expert', 'AstroTerm'): 1,
         ('cosmos', 'AstroTerm'): 1,
         ('galaxy', 'AstroTerm'): 2,
         ('interstellar gas', 'AstroTerm'): 1,
         ('dust', 'AstroTerm'): 1,
         ('star', 'AstroTerm'): 1,
         ('universe', 'AstroTerm'): 2,
         ('Black Hole', 'AstroTerm'): 1,
         ('black hole', 'AstroTerm'): 1,
         ('planet', 'AstroTerm'): 1,
         ('astronomy', 'AstroTerm'): 1,
         ('light', 'AstroTerm'): 2,
         ('collaboration', 'AstroTerm'): 1,
         ('telescope', 'AstroTerm'): 2,
         ('astrophysics', 'AstroTerm'): 1,
         ('Southern Hemisphere', 'AstroTerm'): 1})

In [ ]:
test_text = input("Enter your testing text: ")

In [ ]:
#displacy.render(matched_sents, style="ent", manual=True)